In [2]:
!pip install simpletransformers -q


In [3]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np 
import shutil
import sys
import requests


In [4]:
df=pd.read_csv('/kaggle/input/tf-train/teknofest_train_final (1).csv',delimiter='|')

In [5]:
url = "https://cryptic-oasis-68424.herokuapp.com/preprocess?tr_chars=false&acc_marks=true&punct=true&lower=true&offensive=false&norm_numbers=true&remove_numbers=false&remove_spaces=true&remove_stopwords=false&min_len=4"
texts = df.text.values.tolist()
preprocess_response = requests.post(url, json={"texts": texts})
processed_text = preprocess_response.json()['result']
df.text = processed_text
print(df[df.text == ''].sum())
df = df[df['text'] != '']
print(df.head())
print(df[df.text == ''].sum())

id              e2d954b7-266d-43be-845e-015a8ecf1241697c1629-d...
text                                                             
is_offensive                                                   79
target          RACISTRACISTRACISTRACISTSEXISTOTHEROTHEROTHERR...
dtype: object
                                     id  \
0  81c11060-a240-4d54-841b-9e2916039e85   
1  be80ebbf-b322-4c3b-afa1-94932ea80731   
2  f99e2513-83ed-4076-ac72-b9e2cff3f049   
3  83ed2b2e-b815-4f36-9fc4-80a9050cf2d0   
4  d93e05f7-bfdd-4cdb-99d8-3048761b30ff   

                                                text  is_offensive     target  
0                                        çürük dişli             1     INSULT  
1  bu adamın islama ve müslümanlara verdiği zarar...             1     RACIST  
2                               erkekler zora gelmez             1     SEXIST  
3  utanmazın götüne kazık sokmuşlar bu tıkırtı ne...             1  PROFANITY  
4       otomasyon sistemlerine doğrudan bağlanabilir           

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12467 entries, 0 to 12616
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            12467 non-null  object
 1   text          12467 non-null  object
 2   is_offensive  12467 non-null  int64 
 3   target        12467 non-null  object
dtypes: int64(1), object(3)
memory usage: 487.0+ KB


In [7]:
df_dmy=pd.get_dummies(df.target)

In [8]:
df=pd.concat([df.text, df_dmy], axis=1)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12467 entries, 0 to 12616
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       12467 non-null  object
 1   INSULT     12467 non-null  uint8 
 2   OTHER      12467 non-null  uint8 
 3   PROFANITY  12467 non-null  uint8 
 4   RACIST     12467 non-null  uint8 
 5   SEXIST     12467 non-null  uint8 
dtypes: object(1), uint8(5)
memory usage: 255.7+ KB


In [10]:
target_list=['INSULT','OTHER','PROFANITY','RACIST','SEXIST']

In [11]:
train_df=df.sample(frac=0.70,random_state=52025).reset_index(drop=True)
val_df=df.drop(train_df.index).reset_index(drop=True)

test_df=val_df.sample(frac=0.25,random_state=52025).reset_index(drop=True)
val_df=val_df.drop(test_df.index).reset_index(drop=True)

In [12]:
val_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2805 entries, 0 to 2804
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       2805 non-null   object
 1   INSULT     2805 non-null   uint8 
 2   OTHER      2805 non-null   uint8 
 3   PROFANITY  2805 non-null   uint8 
 4   RACIST     2805 non-null   uint8 
 5   SEXIST     2805 non-null   uint8 
dtypes: object(1), uint8(5)
memory usage: 35.7+ KB


In [13]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8727 entries, 0 to 8726
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       8727 non-null   object
 1   INSULT     8727 non-null   uint8 
 2   OTHER      8727 non-null   uint8 
 3   PROFANITY  8727 non-null   uint8 
 4   RACIST     8727 non-null   uint8 
 5   SEXIST     8727 non-null   uint8 
dtypes: object(1), uint8(5)
memory usage: 110.9+ KB


In [14]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 935 entries, 0 to 934
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       935 non-null    object
 1   INSULT     935 non-null    uint8 
 2   OTHER      935 non-null    uint8 
 3   PROFANITY  935 non-null    uint8 
 4   RACIST     935 non-null    uint8 
 5   SEXIST     935 non-null    uint8 
dtypes: object(1), uint8(5)
memory usage: 12.0+ KB


In [15]:
train_df[target_list].values

array([[0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       ...,
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0]], dtype=uint8)

In [16]:
#bazı hyperparametreler..
MAX_LEN=40
TRAIN_BATCH_SIZE=32
VALID_BATCH_SIZE=32
EPOCHS=7
LR=5e-5
LM = "dbmdz/bert-base-turkish-128k-uncased"

In [17]:
from transformers import BertTokenizer, BertModel

In [18]:
tokenizer = BertTokenizer.from_pretrained(LM, do_lower_case=True, use_cuda=torch.cuda.is_available())
tokenizer

BertTokenizer(name_or_path='dbmdz/bert-base-turkish-128k-uncased', vocab_size=128000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [19]:
exm=" elif ben sana ne demiştim kızım, çocuğu uyuturken uyuya kalma demedim mi # ? "
encodings=tokenizer.encode_plus(exm,
                                add_special_tokens=True,
                                max_length=MAX_LEN,
                                padding='max_length',
                                truncation=True,
                                return_attention_mask=True,
                                return_tensors='pt')

In [20]:
encodings

{'input_ids': tensor([[    2, 12626,  2105,  3247,  2073, 71015,  3180,  9946,    16, 89973,
         27716,  5551, 21660,  6034, 30094,  2477,     7,    35,     3,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [21]:
train_df['text'][0]
print(train_df.values)

[['ben canım sevgilimi ararım' 0 1 0 0 0]
 ['sendeki sinsilik yılanda yok' 1 0 0 0 0]
 ['sikik beyinsiz' 0 0 1 0 0]
 ...
 ['senin kadar çirkin bir insan görmediğime yemin edebilirim ama ıspatlayamam'
  1 0 0 0 0]
 ['oldukça memnun kaldığımı ve çoğu kişiye özellikle fiyat performansı açısından önerdiğimi belirteyim'
  0 1 0 0 0]
 ['seni isteyen kimse kalmadı' 1 0 0 0 0]]


In [22]:
target_list

['INSULT', 'OTHER', 'PROFANITY', 'RACIST', 'SEXIST']

In [23]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['text']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [24]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)

In [25]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [26]:
print(len(train_data_loader))
print(len(val_data_loader))

273
88


In [27]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [28]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()


In [29]:
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [30]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('dbmdz/bert-base-turkish-128k-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 5)#çünkü 5 label var
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at dbmdz/bert-base-turkish-128k-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(128000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [31]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.AdamW(params =  model.parameters(), lr=LR, eps = 1e-7)

In [32]:
val_targets=[]
val_outputs=[]
     

In [33]:
def train_model(n_epochs, training_loader, validation_loader, model, 
                optimizer, checkpoint_path, best_model_path):
  print(len(training_loader))
  print(len(validation_loader))
  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf
   
 
  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)
    
    print('############# Epoch {}: Training End     #############'.format(epoch))
    
    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################    
    # validate the model #
    ######################
 
    model.eval()
   
    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      print(train_loss)
      valid_loss = valid_loss/len(validation_loader)
      print(valid_loss)
      # print training/validation statistics 
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
      
      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
        
        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)
        
      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [34]:
ckpt_path = "/kaggle/working/v1_curr_ckpt"
best_model_path = "/kaggle/working/v1_best_model.pt"

#tokenizer.save_pretrained("/content/drive/MyDrive/Nane&Limon/2023 DDI Yarışma dokümantasyonu/bigscience_t0_tokenizer")
#model.save_pretrained("/content/drive/MyDrive/Nane&Limon/2023 DDI Yarışma dokümantasyonu/bigscience_t0_model")

In [35]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)

273
88
############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
0.000664835443394848
0.0008608391433896511
Epoch: 1 	Avgerage Training Loss: 0.000665 	Average Validation Loss: 0.000861
Validation loss decreased (inf --> 0.000861).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
0.00022600194156305043
0.0006255817367245006
Epoch: 2 	Avgerage Training Loss: 0.000226 	Average Validation Loss: 0.000626
Validation loss decreased (0.000861 --> 0.000626).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training 

In [36]:
from sklearn.metrics import f1_score

test_dataset = CustomDataset(test_df, tokenizer, MAX_LEN)
test_data_loader = torch.utils.data.DataLoader(test_dataset, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

# modeli test verisi ile değerlendirme
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
  for data in test_data_loader:
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    token_type_ids = data["token_type_ids"].to(device)
    targets = data['targets'].to(device)
    outputs = model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(outputs).cpu().detach().numpy().tolist()
    y_true.extend(targets.cpu().numpy())
    y_pred.extend(np.round(final_output))
      
# F1 score macro
f1_macro = f1_score(y_true, y_pred, average='macro')
print("F1 score (macro) : {:.5f}%".format(f1_macro*100))


F1 score (macro) : 97.55665%


In [37]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = BertTokenizer.from_pretrained(LM)
model = BertModel.from_pretrained(LM)

# Modelinizi ve tokenizer'ı yükleyin

torch.load(best_model_path, map_location=torch.device('cpu'))
print(type(model)) # dict dönüyor

# Örnek bir girdi oluşturun
exm = "elif ben sana ne demiştim kızım, çocuğu uyuturken uyuya kalma demedim mi # ?"
example = 'sıska bacaklı göt adam'

MAX_LEN = 128  # örnek bir maksimum girdi uzunluğu
# Örnek girdiyi modeli kullanarak tahmin edin
model.eval()

#girdi tensoru oluştur
encodings = tokenizer.encode_plus(
    exm,
    None,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

# tensörleri atayın
input_ids = encodings['input_ids'].to('cpu', dtype=torch.long)
attention_mask = encodings['attention_mask'].to('cpu', dtype=torch.long)
token_type_ids = encodings['token_type_ids'].to('cpu', dtype=torch.long)

# Tahmin yapın
output = model(input_ids, attention_mask, token_type_ids)

# Tahminleri ekrana yazdırın
print(output)


Some weights of the model checkpoint at dbmdz/bert-base-turkish-128k-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<class 'transformers.models.bert.modeling_bert.BertModel'>
BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-1.8387, -1.1023, -0.8242,  ...,  0.1805,  0.6291,  0.1831],
         [-0.9457, -0.0307, -0.0440,  ...,  0.3992,  2.1232,  1.1942],
         [-0.5214, -0.4406, -0.3887,  ..., -0.3835,  1.1899,  1.3980],
         ...,
         [-1.4262, -1.2720, -1.4024,  ...,  0.2835,  1.2293,  0.9535],
         [-1.8355, -1.6825, -0.8395,  ...,  0.4935,  1.6269,  0.0179],
         [-2.2859, -2.2545, -0.4670,  ...,  0.1839,  1.8497, -0.3740]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 1.0000e+00,  1.4090e-01,  9.9942e-01,  1.2597e-01,  1.7111e-02,
         -1.0000e+00, -3.1108e-01, -9.9998e-01, -9.9549e-01,  9.9999e-01,
         -1.1769e-01,  6.1526e-02,  3.7583e-01, -1.0000e+00, -9.9795e-01,
         -6.1912e-01, -2.1775e-01,  1.1792e-01,  8.2704e-01,  9.9979e-01,
         -1.6528e-01,  1.2525e-01,  9.9211e-01, -1.4756e-01,  3.1955e-01,
        

In [38]:
model_save = model.config.to_json_file('/kaggle/working/config.json') # model config save

In [40]:
tokenizer.save_pretrained('bert-turkish-multi-label-tokenizer')
model.save_pretrained('bert-turkish-multi-label-model')